In [ ]:
%config InlineBackend.figure_formats = {"retina", "png"}
# %matplotlib notebook

import logging
logging.basicConfig(level=logging.INFO)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["font.size"] = 14

import superscreen as sc

from huber_squid import huber_squid, huber_geometry

In [ ]:
device = huber_squid(151)

In [ ]:
device.make_mesh(min_triangles=12000, optimesh_steps=200)

In [ ]:
ax = device.plot_mesh()
ax = device.plot_polygons(ax=ax, color='k', lw=2, legend=False)
_ = ax.set_title(
    f"Mesh: {device.points.shape[0]} points, "
    f"{device.triangles.shape[0]} triangles"
)
ax.figure.set_size_inches(8,8)

In [ ]:
applied_field = sc.sources.ConstantField(0.0)

circulating_currents = {
    "fc_center": "1 mA"
}

solutions = sc.solve(
    device=device,
    applied_field=applied_field,
    circulating_currents=circulating_currents,
    field_units="Phi_0/um**2",
    current_units="uA",
    iterations=11,
    coupled=True,
)
solution = solutions[-1]

In [ ]:
records = []
for s in solutions:
    records.append(s.polygon_flux(units="Phi_0", with_units=False))
df = pd.DataFrame.from_records(records)
df.index.name = "Iteration"

In [ ]:
display(df)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
ax.grid(True)
for col in df.columns:
    ys = df[col].values
    ys = np.abs(np.diff(ys))[1:] / np.abs(ys)[1:-1]
    ax.plot(np.arange(len(ys)) + 1, ys, 'o--', label=col)
ax.set_ylabel("Fractional change in flux\n$(\\Phi_{(i)} -\\Phi_{(i-1)}) / \\Phi_{(i-1)}$", fontsize=14)
ax.set_xlabel("Iteration", fontsize=14)
ax.set_yscale("log")
ax.set_xticks(np.arange(len(ys)) + 1)
ax.legend(loc=0)
ax.set_title("Convergence of calculated flux")
fig.tight_layout()

In [ ]:
flux = solutions[-1].polygon_flux()
I_circ = device.ureg(circulating_currents['fc_center'])
# print(f"{flux['pl_hull'].to('Phi_0'):.3e~P}")
print(
    f"Calculated FC - PL mutual inductance:"
    f"\n\t{(flux['pl_hull'] / I_circ).to('Phi_0/A'):.3f~P} = "
    f"{(flux['pl_hull'] / I_circ).to('pH'):.3f~P}"
)

In [ ]:
for solution in solutions[-3:]:
    fig, axes = sc.plot_fields(
        solution,
        layers=["W1", "W2"],
        units="Phi_0/um**2",
        cmap="coolwarm",
        figsize=(9,5),
        grid_shape=(1000, 1000),
        cross_section_xs=0,
        cross_section_angle=0,
        vmin=-0.3/2, vmax=0.3/2,
    )
    for ax in axes:
        device.plot_polygons(ax=ax, legend=False, color='k', lw=0.75)

In [ ]:
for solution in solutions[-3:]:
    fig, axes = sc.plot_currents(
        solution,
        figsize=(13, 5),
        streamplot=False,
        units="mA/um",
        cmap="inferno",
        cross_section_xs=0,
        cross_section_angle=0,
    )
    for ax in axes:
        device.plot_polygons(ax=ax, legend=False, color='w', lw=0.75, alpha=0.75)

In [ ]:
img = plt.imread("huber-image.png")
scale = np.where(img[385][:,0] == 1)[0].size / 5 # pixels per micron

origin = x0, y0 = (465, 610)

In [ ]:
polygons = {
    name: scale * sc.geometry.rotate(points, 0)
    for name, points in huber_geometry(interp_points=101).items()
}

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
ax.set_aspect("equal")
im = ax.imshow(img[:-200])
ax.set_yticks([])
ax.set_xticks([])

for name, points in polygons.items():
    if name in ["pl_hull", "fc_shield"]:
        continue
    xs, ys = points.T
    xs = -xs
    xs = xs + x0
    ys = ys + y0
    ax.plot(xs, ys, color='w', alpha=0.8, lw=2)
    
_ = ax.set_xlim(0, img.shape[1]-1)